# T1497.001 - System Checks
Adversaries may employ various system checks to detect and avoid virtualization and analysis environments. This may include changing behaviors based on the results of checks for the presence of artifacts indicative of a virtual machine environment (VME) or sandbox. If the adversary detects a VME, they may alter their malware to disengage from the victim or conceal the core functions of the implant. They may also search for VME artifacts before dropping secondary or additional payloads. Adversaries may use the information learned from [Virtualization/Sandbox Evasion](https://attack.mitre.org/techniques/T1497) during automated discovery to shape follow-on behaviors.(Citation: Deloitte Environment Awareness)

Specific checks will vary based on the target and/or adversary, but may involve behaviors such as [Windows Management Instrumentation](https://attack.mitre.org/techniques/T1047), [PowerShell](https://attack.mitre.org/techniques/T1059/001), [System Information Discovery](https://attack.mitre.org/techniques/T1082), and [Query Registry](https://attack.mitre.org/techniques/T1012) to obtain system information and search for VME artifacts. Adversaries may search for VME artifacts in memory, processes, file system, hardware, and/or the Registry. Adversaries may use scripting to automate these checks  into one script and then have the program exit if it determines the system to be a virtual environment. 

Checks could include generic system properties such as host/domain name and samples of network traffic. Adversaries may also check the network adapters addresses, CPU core count, and available memory/drive size. 

Other common checks may enumerate services running that are unique to these applications, installed programs on the system, manufacturer/product fields for strings relating to virtual machine applications, and VME-specific hardware/processor instructions.(Citation: McAfee Virtual Jan 2017) In applications like VMWare, adversaries can also use a special I/O port to send commands and receive output. 
 
Hardware checks, such as the presence of the fan, temperature, and audio devices, could also be used to gather evidence that can be indicative a virtual environment. Adversaries may also query for specific readings from these devices.(Citation: Unit 42 OilRig Sept 2018)

## Atomic Tests

### Atomic Test #1 - Detect Virtualization Environment (Linux)systemd-detect-virt detects execution in a virtualized environment.
At boot, dmesg stores a log if a hypervisor is detected.
**Supported Platforms:** linux
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `sh`
```sh
if (systemd-detect-virt) then echo "Virtualization Environment detected"; fi;
if (sudo dmidecode | egrep -i 'manufacturer|product|vendor' | grep -iE 'Oracle|VirtualBox|VMWare|Parallels') then echo "Virtualization Environment detected"; fi;
```

In [ ]:
Invoke-AtomicTest T1497.001 -TestNumbers 1

### Atomic Test #2 - Detect Virtualization Environment (FreeBSD)Detects execution in a virtualized environment.
At boot, dmesg stores a log if a hypervisor is detected.
**Supported Platforms:** linux
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `sh`
```sh
if [ "$(sysctl -n hw.hv_vendor)" != "" ]; then echo "Virtualization Environment detected"; fi
```

In [ ]:
Invoke-AtomicTest T1497.001 -TestNumbers 2

### Atomic Test #3 - Detect Virtualization Environment (Windows)Windows Management Instrumentation(WMI) objects contains system information which helps to detect virtualization. This command will specifically attempt to get the CurrentTemperature value from this object and will check to see if the attempt results in an error that contains the word supported. This is meant to find the result of Not supported, which is the result if run in a virtual machine
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
$error.clear()
Get-WmiObject -Query "SELECT * FROM MSAcpi_ThermalZoneTemperature" -ErrorAction SilentlyContinue
if($error) {echo "Virtualization Environment detected"}
```

In [ ]:
Invoke-AtomicTest T1497.001 -TestNumbers 3

#### Cleanup: 
```powershell
$error.clear()
```

In [ ]:
Invoke-AtomicTest T1497.001 -TestNumbers 3 -Cleanup

### Atomic Test #4 - Detect Virtualization Environment (MacOS)ioreg contains registry entries for all the device drivers in the system. If it's a virtual machine, one of the device manufacturer will be a Virtualization Software.
**Supported Platforms:** macos
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `sh`
```sh
if (ioreg -l | grep -e Manufacturer -e 'Vendor Name' | grep -iE 'Oracle|VirtualBox|VMWare|Parallels') then echo 'Virtualization Environment detected'; fi;
```

In [ ]:
Invoke-AtomicTest T1497.001 -TestNumbers 4

### Atomic Test #5 - Detect Virtualization Environment via WMI Manufacturer/Model Listing (Windows)Windows Management Instrumentation(WMI) objects contain system information which helps to detect virtualization. This test will get the model and manufacturer of the machine to determine if it is a virtual machine, such as through VMware or VirtualBox. 
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
$Manufacturer = Get-WmiObject -Class Win32_ComputerSystem | select-object -expandproperty "Manufacturer"
$Model = Get-WmiObject -Class Win32_ComputerSystem | select-object -expandproperty "Model"
if((($Manufacturer.ToLower() -eq "microsoft corporation") -and ($Model.ToLower().contains("virtual"))) -or ($Manufacturer.ToLower().contains("vmware")) -or ($Model.ToLower() -eq "virtualbox")) {write-host "Virtualization environment detected!"} else {write-host "No virtualization environment detected!"}
```

In [ ]:
Invoke-AtomicTest T1497.001 -TestNumbers 5

## Detection
Virtualization/sandbox related system checks will likely occur in the first steps of an operation but may also occur throughout as an adversary learns the environment. Data and events should not be viewed in isolation, but as part of a chain of behavior that could lead to other activities, such as lateral movement, based on the information obtained. Detecting actions related to virtualization and sandbox identification may be difficult depending on the adversary's implementation and monitoring required. Monitoring for suspicious processes being spawned that gather a variety of system information or perform other forms of Discovery, especially in a short period of time, may aid in detection.